In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import kagglehub
import numpy as np
import matplotlib.dates as mdates

: 

In [2]:
file_path = r"C:\Users\simoo\.cache\kagglehub\datasets\camnugent\sandp500\versions\4\all_stocks_5yr.csv"
main_df = pd.read_csv(file_path)
main_df

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL
...,...,...,...,...,...,...,...
619035,2018-02-01,76.84,78.27,76.69,77.82,2982259,ZTS
619036,2018-02-02,77.53,78.12,76.73,76.78,2595187,ZTS
619037,2018-02-05,76.64,76.92,73.18,73.83,2962031,ZTS
619038,2018-02-06,72.74,74.56,72.13,73.27,4924323,ZTS


In [3]:
# Set stock of interest
ticker = 'AAPL'

# Filter on ticker above
stock_df = main_df[main_df['Name']==ticker].copy().reset_index(drop=True)

# Set date column
stock_df['date'] = pd.to_datetime(stock_df['date'])

## Lets go wild with the LSTM

In [5]:

main_df['date'] = pd.to_datetime(main_df['date'])

# Sortera DataFrame efter aktie och datum
main_df = main_df.sort_values(['Name', 'date'])

# Räkna log-returns per aktie
main_df['log_return'] = main_df.groupby('Name')['close'].apply(lambda x: np.log(x) - np.log(x.shift(1)))


In [8]:
main_df

,date,open,high,low,close,volume,Name,log_return
71611,2013-02-08,45.07,45.35,45.00,45.08,1824755,A,NaN
71612,2013-02-11,45.17,45.18,44.45,44.60,2915405,A,-0.010705
71613,2013-02-12,44.81,44.95,44.50,44.62,2373731,A,0.000448
71614,2013-02-13,44.81,45.24,44.68,44.75,2052338,A,0.002909
71615,2013-02-14,44.72,44.78,44.36,44.58,3826245,A,-0.003806
...,...,...,...,...,...,...,...,...
619035,2018-02-01,76.84,78.27,76.69,77.82,2982259,ZTS,0.014106
619036,2018-02-02,77.53,78.12,76.73,76.78,2595187,ZTS,-0.013454
619037,2018-02-05,76.64,76.92,73.18,73.83,2962031,ZTS,-0.039179
619038,2018-02-06,72.74,74.56,72.13,73.27,4924323,ZTS,-0.007614


In [9]:
window_size = 20
target_horizon = 5

X_all = []
y_all = []

# Loopa över alla aktier
for name, group in main_df.groupby('Name'):
    returns = group['log_return'].dropna().values

    # Skapa sliding windows
    for i in range(len(returns) - window_size - target_horizon):
        x_seq = returns[i : i + window_size]
        y_vol = np.std(returns[i + window_size : i + window_size + target_horizon])
        
        X_all.append(x_seq)
        y_all.append(y_vol)


In [ ]:
import torch

: 

In [12]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

class VolatilityDataset(Dataset):
    def __init__(self, returns, window_size=20, target_horizon=5):
        self.X = []
        self.y = []
        for i in range(len(returns) - window_size - target_horizon):
            x_seq = returns[i:i+window_size]
            y_target = np.std(returns[i+window_size:i+window_size+target_horizon])
            self.X.append(x_seq)
            self.y.append(y_target)
        self.X = torch.tensor(self.X, dtype=torch.float32).unsqueeze(-1)
        self.y = torch.tensor(self.y, dtype=torch.float32).unsqueeze(-1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

ModuleNotFoundError: No module named 'torch'

In [ ]:
X_tensor = torch.tensor(X_all, dtype=torch.float32).unsqueeze(-1)
y_tensor = torch.tensor(y_all, dtype=torch.float32).unsqueeze(-1)

dataset = VolatilityDataset(X_tensor, y_tensor)


In [ ]:
class LSTMVolModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=32, num_layers=1):
        super(LSTMVolModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]  # sista tidpunkten
        out = self.linear(out)
        return out

In [ ]:
dataset = VolatilityDataset(returns, window_size=20, target_horizon=5)
train_size = int(0.8 * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
model = LSTMVolModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
for epoch in range(20):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.6f}")

In [ ]:
model.eval()
all_preds, all_targets = [], []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        y_pred = model(X_batch)
        all_preds.append(y_pred.numpy())
        all_targets.append(y_batch.numpy())

all_preds = np.vstack(all_preds)
all_targets = np.vstack(all_targets)

mse = mean_squared_error(all_targets, all_preds)
print(f"\n✅ Test MSE: {mse:.6f}")

In [ ]:
plt.plot(all_targets[:100], label='Realized Volatility')
plt.plot(all_preds[:100], label='Predicted (LSTM)')
plt.legend()
plt.title("Prediktion av realiserad volatilitet")
plt.show()